In [1]:
import requests
import os
from langchain.docstore.document import Document
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI 
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_chroma import Chroma
import re

from dotenv import load_dotenv
load_dotenv()

base_url = os.environ["SERVICENOW_BASEURL"]
user = os.environ["SERVICENOW_USERNAME"]
pwd = os.environ["SERVICENOW_PASSWORD"]
headers = {"Content-Type":"application/json","Accept":"application/json"}


def get_specific_catalog_item(sys_id):

    url = f"{base_url}/sn_sc/servicecatalog/items/{sys_id}"
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print("Status:", response.status_code, "Headers:", response.headers, "Error Response:",response.json())
        exit()
    data = response.json()
    print(data)
    return data




def get_table_response(reference):
    url = f"{base_url}/now/table/{reference}?sysparm_limit=1"
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print("Status:", response.status_code, "Headers:", response.headers, "Error Response:",response.json())
        exit()
    data = response.json()
    return data


def get_table_sysid(reference, table_field, value_for_query):
    if("@" in value_for_query):
        value_for_query = value_for_query.replace("@","%40")
    sysparm_query = f"{table_field}%3D{value_for_query}" 

    url = f"{base_url}/now/table/{reference}?sysparm_query={sysparm_query}&sysparm_limit=1"
    # print(url)
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print("Status:", response.status_code, "Headers:", response.headers, "Error Response:",response.json())
        exit()

    data = response.json()
    return data


def get_catalog_item_variables(sys_id):
    url = f"{base_url}/sn_sc/servicecatalog/items/{sys_id}/variables"
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print("Status:", response.status_code, "Headers:", response.headers, "Error Response:",response.json())
    print(response)
    data = response.json()
    return data

def get_table_values(reference):
    url = f"{base_url}/now/table/{reference}"
    response = requests.get(url, auth=(user, pwd), headers=headers )
    if response.status_code != 200: 
        print("Status:", response.status_code, "Headers:", response.headers, "Error Response:",response.json())
        exit()
    data = response.json()
    return data


In [2]:
from langchain.schema import HumanMessage
import json
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI 

model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

In [3]:
variables_description = {"requested_for": "The user for whom the request is being made. This field is mandatory.",
 "contact_number": "The contact number of the requester. This field is mandatory.",
 "Location": "The location of the requester. This field is optional and dynamically filled based on the 'Requested For' field.",
 "project_name": "The project name under which the software service is requested. This field is optional and dynamically filled based on the 'Requested For' field.",
 "business_service": "The type of business service required. This field is mandatory. Choices are: Software Uninstallation, Software Installation, OS Upgrade. UI Policy: If the business service is 'Software Installation,' the following fields become mandatory and visible: request_type, software_name, add_software, duration.",
 "software_name": "The name of the software for which the service is requested. This field is optional. Choices are: Opera, Stripe SDK, Android SDK, Dropbox SDK, Canva, OpenCV SDK, Java, Perl, Atom, Telegram Bot SDK, Node.js SDK, Adobe Premiere Pro, Hadoop, Agora SDK, GTK+ SDK, OneSignal SDK, NVivo, Mixpanel SDK, Swift SDK, JavaScript, Visual Studio Code, Adobe Illustrator, Prolog, Skype, VB.NET, GameMaker Studio, Slack SDK, Swift, Ruby, Nim, Tcl, Elixir, Zoom SDK, Scikit-learn, Unreal Engine, Google Chrome, Xero, Final Draft, ARCore, .NET SDK, Affinity Designer, CorelDRAW, MATLAB, Stata, Keras, Cocoa Touch, Maya, Azure SDK, TypeScript, Box SDK, Kubernetes, Amplitude SDK, VHDL, TensorFlow, Jupyter Notebook, Groovy, Microsoft Edge, Spring Framework, Less, Microsoft Word, Google Cloud SDK, RStudio, Apache Spark, Overleaf, Jira, Oracle Database, Sage 50, Terraform, Dart, Sublime Text, Realm SDK, SAS, Adobe After Effects, React Native SDK, Redis, MySQL, Eclipse, NetBeans, 3ds Max, Twitter SDK, R, Fortran, Pascal, CSS, Scheme, Notepad++, Windows SDK, Common Lisp, Discord SDK, Android Studio, Microsoft Excel, LaTeX, MongoDB, Segment SDK, SolidWorks, Haskell, Cassandra, Flutter SDK, Xcode, Rust, PostgreSQL, SAP, TensorFlow SDK, Xamarin SDK, Vagrant, QuickBooks, PyCharm, GitHub, EndNote, Salesforce SDK, iOS SDK, Celtx, Salesforce, PayPal SDK, SQLite, C++, Google Meet, Zoom, Ada, Assembly Language, Unreal Engine SDK, SketchUp, Java Development Kit (JDK), Firebase SDK, Kotlin, Erlang, Adobe Acrobat Reader, Microsoft Teams, Apache Cordova SDK, Power BI, Mendeley, PowerShell, Verilog, Ansible, Minitab, Go (Golang), Sketch, Clojure, Unity, Facebook SDK, Intercom SDK, SPSS, Safari, Objective-C, ARKit, Grammarly, Julia, Crystal, AutoCAD, COBOL, FreshBooks, Scrivener, InVision, Kotlin SDK, Sass, Mathematica, Mozilla Firefox, PHP, SQL, D, Sentry SDK, Adobe Photoshop, Trello, QlikView, Zotero, HTML, Microsoft PowerPoint, Tableau, Lua, Python, Scala, Google Maps SDK, Electron SDK, NoSQL, C#, New Relic SDK, Affinity Photo, F#, Docker, Asana, Twilio SDK, Procreate, AWS SDK, Bitbucket, GitLab, Slack, Unity SDK, IntelliJ IDEA, Microsoft SQL Server, Figma, PyTorch, Shell Scripting (Bash), SAP SDK, C, Qt SDK, Jenkins, Blender, RPG Maker.",
 "add_software": "Checkbox to add another software. This field is optional. UI Policy: If the user checks 'Add Software', the fields software_name1, request_type1, duration1, add_software1 become mandatory and visible.",
 "request_type": "The type of request. This field is optional and dynamically filled based on the 'Software Name' field.",
 "duration": "The duration for which the software service is required. This field is optional. Choices are: None, 30 days, 60 days, 90 days, 180 days, 360 days.",
 "software_name1": "The name of the additional software for which the service is requested. This field is optional. Choices are the same as the primary 'Software Name' field.",
 "add_software1": "Checkbox to add yet another software. This field is optional. UI Policy: If the user checks 'Add Software (1)', the fields software_name2, request_type2, duration2, add_software2 become mandatory and visible.",
 "request_type1": "The type of request for the additional software. This field is optional and dynamically filled based on the 'Software Name (1)' field.",
 "duration1": "The duration for which the additional software service is required. This field is optional. Choices are the same as the primary 'Duration' field.",
 "software_name2": "The name of the third software for which the service is requested. This field is optional. Choices are the same as the primary 'Software Name' field.",
 "add_software2": "Checkbox to add a fourth software. This field is optional.",
 "request_type2": "The type of request for the third software. This field is optional and dynamically filled based on the 'Software Name (2)' field.",
 "duration2": "The duration for which the third software service is required. This field is optional. Choices are the same as the primary 'Duration' field.",
 "software_name3": "The name of the fourth software for which the service is requested. This field is optional. Choices are the same as the primary 'Software Name' field. UI Policy: If the business service is 'Software Uninstallation', this field becomes mandatory and visible.",
 "select_your_host_name": "The host name for the software service. This field is mandatory. Choices are: HostName1, HostName2, HostName3."}

In [4]:
def create_custom_function(variable_info, custom_functions):
    inner_json = {}
    inner_json["type"] = "string"
    inner_json["description"] = variables_description[variable_info["name"]]
    custom_functions[0]["parameters"]["properties"][variable_info["name"]] = inner_json
    return custom_functions


def function_calling_catVar(catalog_variables):
    custom_functions = [
        {
            "name": "extract_catalog_variables",
            "description": "Get the values from the body of the user query",
            "parameters": {
                "type": "object",
                "properties": {}
            }
        }
        ]

    for variable_info in catalog_variables:
        if "children" in variable_info:
            for child_var_info in variable_info["children"]:
                # print(child_var_info)
                custom_functions = create_custom_function(child_var_info, custom_functions)
        else:
            custom_functions = create_custom_function(variable_info, custom_functions)
    return custom_functions

In [5]:
def get_all_variables_List(catalog_item_variable):
    def extract_variables(variables):
        variable_List = []
        for variable_info in variables:
            if variable_info["type"] == 24:
                continue
            if "children" in variable_info:
                variable_List.extend(extract_variables(variable_info["children"]))
            else:
                variable_List.append(variable_info)
        return variable_List
    return extract_variables(catalog_item_variable)


def set_reference_value(variable):
    
    if(variable["type"] == 8 and variable["dynamic_value_field"] == ""):
        reference = variable["reference"]
        table_content = get_table_values(reference=reference)
        names_in_table = []
        for content in table_content["result"]:
            if("u_name" in content):
                names_in_table.append(content["u_name"])
        variable["reference_values"] = ", ".join(names_in_table)
        return variable
    
    elif (variable["type"] == 5 or variable["type"] == 18):
        choiceList = []
        for choice in variable["choices"]:
            choiceList.append(choice["value"])
        variable["reference_values"] = ", ".join(choiceList)
        return variable

    elif(variable["value"] != ""):
        variable["displayvalue"] = variable["value"]
        return variable

    else:
        return variable

def fetch_ui_actions(action_field, catalog_variables):
    ui_action_variable_list = []
    for action_variables in action_field:
        action_variable_id = action_variables["name"].split(":")[1] 
        for variable in catalog_variables:
            # print(f"{variable["id"]} = {action_variable_id}")
            if(variable["id"] == action_variable_id):
                ui_action_variable_list.append({"name": variable["name"], "mandatory": action_variables["mandatory"]})
    
    return ui_action_variable_list


def fetch_ui_policy(catalog_item, catalog_variables):
    final_ui_condition_action = []
    for catalog_ui_policy in catalog_item["result"]["ui_policy"]:
        for condition_field in catalog_ui_policy["conditions"]:
            action_field = catalog_ui_policy["actions"]
            ui_action_variable_list = fetch_ui_actions(action_field, catalog_variables)
            variable_id = condition_field["field"].split(":")[1]
            for variable in catalog_variables:
                if variable["id"] == variable_id:
                    ui_condition = {"condition_variable_name":variable["name"],"condition_variable_value": condition_field["value"], "condition_operation": condition_field["oper"], "ui_actions": ui_action_variable_list}
                    final_ui_condition_action.append(ui_condition)
    return final_ui_condition_action


def arrange_api_request(parse_variable_details,variables_template):
    for key in parse_variable_details:
        if key in variables_template:
            variables_template[key] = parse_variable_details[key]
    return variables_template


def get_valuefor_reqfor(variable_info, variables_template, fetched_variables):
        reference = variable_info["reference"]
        sysparm_for_query = "user_name"
        value_for_query = os.environ["SERVICENOW_USERNAME"]
        # print(reference + sysparm_for_query + value_for_query)
        sys_id = get_table_sysid(reference, sysparm_for_query, value_for_query)["result"][0]["sys_id"]
        variables_template[variable_info["name"]] = sys_id
        fetched_variables[variable_info["name"]] = sys_id
        return fetched_variables, variables_template


def get_valuefor_reference(variable_info, variables_template, fetched_variables):
    reference = variable_info["reference"] # u_software

    sample_table_response = get_table_response(reference)
    if "u_name" in sample_table_response["result"][0]:
        sysparm_for_query = "u_name" # name
    elif "name" in sample_table_response["result"][0]:
        sysparm_for_query = "name"
    
    # print(f"type 8 = {variable_info["name"]}")
    if(variable_info["name"] in fetched_variables):
        # print(f"type 8 = {variable_info["name"]}")
        value_for_query = fetched_variables[variable_info["name"]] # python
        # print(reference + sysparm_for_query + value_for_query)
        sys_id = get_table_sysid(reference, sysparm_for_query, value_for_query)["result"][0]["sys_id"]
        # print(sys_id)
        # print(f"{variables_template[variable_info["name"]]} = {sys_id}")
        variables_template[variable_info["name"]] = sys_id
        
    return fetched_variables, variables_template



def is_valid_sys_id(value):
    return bool(re.fullmatch(r"[0-9a-fA-F]{32}", value))




def get_valuefor_dynamicvalues(variables_List, variable_info, variables_template, fetched_variables):
    for variables in variables_List:
        if(variables["id"] == variable_info["dynamic_value_field"]):
            reference = variables["reference"]
            sysparm_for_query = "sys_id"# variable_info["dynamic_value_dot_walk_path"]
            value_for_query = variables_template[variables["name"]]
        
            # print(" reference =  "+ reference +" sysparm_for_query =  "+ sysparm_for_query +" value_for_query = "+ value_for_query)
            # print([variable_info["dynamic_value_dot_walk_path"]])
            requireddetails = get_table_sysid(reference, sysparm_for_query, value_for_query)
            if("result" in requireddetails):
                resultValue = requireddetails["result"]
                if(isinstance(resultValue, list)):
                    if(0<len(resultValue)):
                        requireddetails = resultValue[0][variable_info["dynamic_value_dot_walk_path"]]
            finalDetail = ""
            # print(requireddetails)
            if("value" in requireddetails):
                finalDetail = requireddetails["value"]
            else:
                finalDetail = requireddetails
            
            variables_template[variable_info["name"]] = finalDetail
    return fetched_variables, variables_template




def assign_complex_variables(fetched_variables, variables_template, variables_List):
    variables_template = arrange_api_request(fetched_variables, variables_template)
    for variable_info in variables_List:
        # print(variable_info["name"])
        if(variable_info["type"] == 31):            # for requested_for type
            fetched_variables, variables_template = get_valuefor_reqfor(variable_info, variables_template, fetched_variables)

        elif(variable_info["name"] in fetched_variables and variable_info["type"] == 8):
            fetched_variables, variables_template = get_valuefor_reference(variable_info, variables_template, fetched_variables)

        elif (variable_info["dynamic_value_field"] != ""):
            fetched_variables, variables_template = get_valuefor_dynamicvalues(variables_List, variable_info, variables_template, fetched_variables)
    return fetched_variables, variables_template

def fetch_mandatory_variables(variables_template, ui_policy,variables_List):
    missing_mandatory_variables = []
    for variable in variables_List:
        if(variable["mandatory"] == True and variables_template[variable["name"]] == ""):
            missing_mandatory_variables.append(variable["name"])
    for ui_condition in ui_policy:
        if(ui_condition["condition_operation"] == "="):
            if(variables_template[ui_condition["condition_variable_name"]] == ui_condition["condition_variable_value"]):
                ui_actions = ui_condition["ui_actions"]
                for ui_action in ui_actions:
                    if(ui_action["mandatory"] == "true" and variables_template[ui_action["name"]] == ""):
                        missing_mandatory_variables.append(ui_action["name"])
                    
    return variables_template, missing_mandatory_variables

def get_variable_from_query(user_query, sys_id):
    catalog_item = get_specific_catalog_item(sys_id)
    catalog_variables = catalog_item["result"]["variables"]
    ui_policy = fetch_ui_policy(catalog_item, get_all_variables_List(catalog_variables))
    variables_List = []
    variables_template = {}
    for variable in get_all_variables_List(catalog_variables):
        variable = set_reference_value(variable)
        variables_List.append(variable)
        variables_template[variable["name"]] = variable["displayvalue"]
        
    custom_function = function_calling_catVar(variables_List)
    message = model.predict_messages(
    [HumanMessage(content=f"user query: {user_query}")],
    functions = custom_function
    )

    print(variables_List)
    # print(custom_function)
    # print(json.dumps(message.__dict__))

    if message.additional_kwargs != {}:
        # print("The "additional_kwargs" attribute exists.")
        parse_variable_details = json.loads(message.additional_kwargs["function_call"]["arguments"])
        parse_variable_details,variables_template = assign_complex_variables(parse_variable_details,variables_template, variables_List)
        variables_template, missing_mandatory_variables = fetch_mandatory_variables(variables_template, ui_policy,variables_List)
            
    return parse_variable_details, json.dumps(variables_template), ui_policy, missing_mandatory_variables

In [6]:
get_variable_from_query("I need to install python, java and dot net softwares", "50572ffac3405a1068d8b132b4013177")

{'result': {'short_description': 'Request for Software Services', 'kb_article': '', 'icon': 'images/service_catalog/generic_small.gifx', 'description': '<div>\r\n<ul><li>Please select Business Function/Request Type to display delivery time</li></ul>\r\n</div>', 'availability': 'on_desktop', 'mandatory_attachment': False, 'request_method': '', 'type': 'catalog_item', 'visible_standalone': True, 'sys_class_name': 'sc_cat_item', 'sys_id': '50572ffac3405a1068d8b132b4013177', 'content_type': '', 'order': 0, 'make_item_non_conversational': False, 'owner': '6816f79cc0a8016401c5a33be04be441', 'show_price': False, 'show_quantity': True, 'picture': '', 'url': '', 'catalogs': [{'sys_id': 'e0d08b13c3330100c8b837659bba8fb4', 'active': True, 'title': 'Service Catalog'}], 'name': 'Software Services', 'show_wishlist': True, 'category': {'sys_id': '01d72b3ec3405a1068d8b132b40131ac', 'title': 'Personal Catalogs'}, 'turn_off_nowassist_conversation': False, 'show_delivery_time': True, 'categories': [], 'v

c:\Users\2000081253\Desktop\Work\AI-Copilot-Ticket-Assistant\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[{'active': True, 'label': 'Requested For', 'dynamic_value_field': '', 'type': 31, 'mandatory': True, 'displayvalue': 'javascript:gs.getUserID();', 'friendly_type': 'requested_for', 'display_type': 'Requested For', 'reference': 'sys_user', 'render_label': True, 'ref_qualifier': 'EQ', 'read_only': False, 'name': 'requested_for', 'attributes': 'edge_encryption_enabled=true', 'id': '9228637ec3405a1068d8b132b4013151', 'value': 'javascript:gs.getUserID();', 'dynamic_value_dot_walk_path': '', 'help_text': '', 'max_length': 0, 'order': 1}, {'active': True, 'label': 'Contact Number', 'dynamic_value_field': '', 'type': 6, 'mandatory': True, 'displayvalue': '', 'friendly_type': 'single_line_text', 'display_type': 'Single Line Text', 'render_label': True, 'read_only': False, 'name': 'contact_number', 'attributes': 'edge_encryption_enabled=true', 'id': 'cfa82f7ec3405a1068d8b132b4013157', 'value': '', 'dynamic_value_dot_walk_path': '', 'help_text': '', 'max_length': 0, 'order': 2}, {'active': True,

({'business_service': 'Software Installation',
  'software_name': 'Python',
  'add_software': 'true',
  'software_name1': 'Java',
  'add_software1': 'true',
  'software_name2': '.NET SDK',
  'requested_for': '6816f79cc0a8016401c5a33be04be441'},
 '{"requested_for": "6816f79cc0a8016401c5a33be04be441", "contact_number": "", "Location": "163c49b037d0200044e0bfc8bcbe5dcc", "project_name": "a581ab703710200044e0bfc8bcbe5de8", "business_service": "effae772c3805a1068d8b132b40131dd", "software_name": "7c29b2afc3441a5068d8b132b4013164", "add_software": "true", "request_type": "Software Installation - Freeware", "duration": "", "software_name1": "3029b2afc3441a5068d8b132b4013165", "add_software1": "true", "request_type1": "Software Installation - Freeware", "duration1": "", "software_name2": "fc29b2afc3441a5068d8b132b401316e", "add_software2": false, "request_type2": "Software Installation - Freeware", "duration2": "", "software_name3": "", "select_your_host_name": ""}',
 [{'condition_variable_nam

In [7]:
def validateParameters(user_query, variable_info):

    if "reference_values" in variable_info:
        preTemplate = f"user_query : {user_query} Please select one of the following options for the {variable_info["name"]} based on the user query: {variable_info["reference_values"]}."
    else:
        preTemplate = f"user_query : {user_query} If {variable_info["name"]} not mentioned in user query then assign {variable_info["displayvalue"]}."

    template = """
    {preTemplate}
    If it is not exist in user query then Extracted value is None.
    Your result should contain only the Extracted value only.
    And no premable or explaination.

    """
    
    prompt_template = PromptTemplate(template=template, input_variables=['preTemplate'])
    chain = prompt_template | model | StrOutputParser()
    response = chain.invoke({"preTemplate": preTemplate})
    return response

validateParameters("Good Morning",{"active":True,"label":"Business Service","dynamic_value_field":"","type":8,"mandatory":True,"displayvalue":"","friendly_type":"reference","display_type":"Reference","reference":"u_business_services","render_label":True,"ref_qualifier":"","read_only":False,"name":"business_service","attributes":"edge_encryption_enabled=true","id":"4d4ba3b2c3805a1068d8b132b401319c","value":"","dynamic_value_dot_walk_path":"","help_text":"","max_length":0,"order":5,"reference_values":"Software Uninstallation, OS Upgrade, Software Installation"})

SyntaxError: f-string: unmatched '[' (444723421.py, line 4)